# Bert2Bert on Wikihow

## Libraries and environment preparation

In [ ]:
#Install essential packages
%%capture
!pip install datasets rouge-score nltk wandb
!pip install transformers==4.11.0
!apt install git-lfs

In [ ]:
#Colab Environment Check for GPU and RAM
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

#GPU check
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Your runtime has 13.6 gigabytes of available RAM

Thu Feb 24 17:43:45 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   66C    P8    10W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                      

In [ ]:
# Make sure your version of Transformers is at 4.11.0
# to run the following code correctly:
import datasets
import transformers

In [ ]:
# Import Wandb 
import os
import wandb
API_KEY = '39991c538626bee25c64d4f8a4c3403dd635537c'
os.environ["WANDB_API_KEY"] = API_KEY

## Loading the dataset

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# import dataset and metrics with huggingface
raw_datasets = datasets.load_dataset("wikihow", "all", "/content/drive/MyDrive/dataset")

Downloading:   0%|          | 0.00/2.97k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.29k [00:00<?, ?B/s]

Using custom data configuration all-6f5101161f12f62f


  0%|          | 0/3 [00:00<?, ?it/s]

Downloading:   0%|          | 0.00/1.27M [00:00<?, ?B/s]

...

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

Dataset wikihow downloaded and prepared to /root/.cache/huggingface/datasets/wikihow/all-6f5101161f12f62f/1.2.0/5343fc81d685acaa086c9cc19eb8706206cd1f8b315792b04c1d7b92091c305e. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['text', 'headline', 'title'],
        num_rows: 157252
    })
    validation: Dataset({
        features: ['text', 'headline', 'title'],
        num_rows: 5599
    })
    test: Dataset({
        features: ['text', 'headline', 'title'],
        num_rows: 5577
    })
})

## Preprocessing the data

In [ ]:
from transformers import BertTokenizerFast

tokenizer = BertTokenizerFast.from_pretrained("bert-base-uncased")
tokenizer.bos_token = tokenizer.cls_token
tokenizer.eos_token = tokenizer.sep_token

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

/usr/local/lib/python3.7/dist-packages/transformers/configuration_utils.py:337: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  "Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 "


In [ ]:
encoder_max_length=512
decoder_max_length=128

def process_data_to_model_inputs(batch):
  # tokenize the inputs and labels
  inputs = tokenizer(batch["text"], padding="max_length", truncation=True, max_length=encoder_max_length)
  outputs = tokenizer(batch["headline"], padding="max_length", truncation=True, max_length=decoder_max_length)

  batch["input_ids"] = inputs.input_ids
  batch["attention_mask"] = inputs.attention_mask
  batch["decoder_input_ids"] = outputs.input_ids
  batch["decoder_attention_mask"] = outputs.attention_mask
  batch["labels"] = outputs.input_ids.copy()

  # because BERT automatically shifts the labels, the labels correspond exactly to `decoder_input_ids`. 
  # We have to make sure that the PAD token is ignored
  batch["labels"] = [[-100 if token == tokenizer.pad_token_id else token for token in labels] for labels in batch["labels"]]

  return batch

In [ ]:
train_data = raw_datasets["train"]
train_data = train_data.map(
    process_data_to_model_inputs, 
    batched=True, 
    remove_columns=["text", "headline", "title"]
)
train_data.set_format(
    type="torch", columns=["input_ids", "attention_mask", "decoder_input_ids", "decoder_attention_mask", "labels"],
)

  0%|          | 0/158 [00:00<?, ?ba/s]

In [ ]:
# bert2bert validation step is high computation cost. We choose about 1/5
val_data = raw_datasets['validation'].select(range(3000))
val_data = val_data.map(
    process_data_to_model_inputs, 
    batched=True, 
    remove_columns=["text", "headline", "title"]
)
val_data.set_format(
    type="torch", columns=["input_ids", "attention_mask", "decoder_input_ids", "decoder_attention_mask", "labels"],
)

  0%|          | 0/3 [00:00<?, ?ba/s]

## Fine-tuning the model

In [ ]:
# Import tokenizer from model checkpoint and print detail
from transformers import EncoderDecoderModel
#bert2bert = EncoderDecoderModel.from_encoder_decoder_pretrained("bert-base-cased", "bert-base-cased", tie_encoder_decoder=True)
bert2bert = EncoderDecoderModel.from_encoder_decoder_pretrained("bert-base-uncased", "bert-base-uncased")

/usr/local/lib/python3.7/dist-packages/transformers/configuration_utils.py:337: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  "Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 "


Downloading:   0%|          | 0.00/420M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertLMHeadModel: ['cls.seq_relationship.bias', 'cls.seq_relations

Setting the special tokens.
`bert-base-cased` does not have a `decoder_start_token_id` or `eos_token_id`, so we will use its `cls_token_id` and `sep_token_id` respectively. 
Also, we should define a `pad_token_id` on the config and make sure the correct `vocab_size` is set.

In [ ]:
# set special tokens
bert2bert.config.decoder_start_token_id = tokenizer.cls_token_id
bert2bert.config.eos_token_id = tokenizer.sep_token_id
bert2bert.config.pad_token_id = tokenizer.pad_token_id

# sensible parameters for beam search
bert2bert.config.vocab_size = bert2bert.config.decoder.vocab_size
bert2bert.config.max_length = 142
bert2bert.config.min_length = 56
bert2bert.config.no_repeat_ngram_size = 3
bert2bert.config.early_stopping = True
bert2bert.config.length_penalty = 2.0
bert2bert.config.num_beams = 4

Define `Seq2SeqTrainer` to compute the metrics from the predictions, and also do a bit of pre-processing to decode the predictions into texts:

In [ ]:
# keep track with wandb
wandb.init(project="bert2bert")

wandb: Currently logged in as: shusunny (use `wandb login --relogin` to force relogin)


In [ ]:
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments

In [ ]:
import nltk
import numpy as np
nltk.download('punkt')

from datasets import load_metric
metric = load_metric("rouge")

def compute_metrics(eval_pred):
    predictions = eval_pred.predictions
    labels = eval_pred.label_ids
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
        
    result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
    # Extract a few results
    result = {key: value.mid.fmeasure * 100 for key, value in result.items()}
    
    return {k: round(v, 4) for k, v in result.items()}

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


Downloading:   0%|          | 0.00/2.16k [00:00<?, ?B/s]

In [ ]:
# set training arguments - these params are not really tuned, feel free to change
batch_size = 8
epochs = 1
training_args = Seq2SeqTrainingArguments(
    output_dir="./",
    evaluation_strategy="steps",
    load_best_model_at_end="eval_loss",
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    predict_with_generate=True,
    logging_steps=1000,  # set to 1000 for full training
    save_steps=2000,  # set to 500 for full training
    eval_steps=2000,  # set to 8000 for full training
    warmup_steps=500,  # set to 2000 for full training
    overwrite_output_dir=True,
    num_train_epochs=epochs,
    save_total_limit=3,
    fp16=True, 
    report_to="wandb",
)

In [ ]:
# Pass into the trainer
trainer = Seq2SeqTrainer(
    model=bert2bert,
    args=training_args,
    train_dataset=train_data,
    eval_dataset=val_data,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

Using amp fp16 backend


In [19]:
trainer.train()

***** Running training *****
  Num examples = 157252
  Num Epochs = 1
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 19657
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"


Step,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum
2000,3.775500,3.553531,22.462800,4.705500,15.931100,15.932700
4000,3.438700,3.328399,25.469000,6.256100,17.943900,17.946000
6000,3.273500,3.190789,25.933500,6.720700,18.472300,18.476700
8000,3.164400,3.091385,28.296600,7.843900,19.966700,19.967900


***** Running Evaluation *****
  Num examples = 3000
  Batch size = 8
Saving model checkpoint to ./checkpoint-2000
/usr/local/lib/python3.7/dist-packages/transformers/configuration_utils.py:337: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  "Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 "
Configuration saved in ./checkpoint-2000/config.json
Model weights saved in ./checkpoint-2000/pytorch_model.bin
tokenizer config file saved in ./checkpoint-2000/tokenizer_config.json
Special tokens file saved in ./checkpoint-2000/special_tokens_map.json
***** Running Evaluation *****
  Num examples = 3000
  Batch size = 8
Saving model checkpoint to ./checkpoint-4000
/usr/local/lib/python3.7/dist-packages/transfo

Step,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum
2000,3.775500,3.553531,22.462800,4.705500,15.931100,15.932700
4000,3.438700,3.328399,25.469000,6.256100,17.943900,17.946000
6000,3.273500,3.190789,25.933500,6.720700,18.472300,18.476700
8000,3.164400,3.091385,28.296600,7.843900,19.966700,19.967900
10000,3.072900,3.012250,28.173200,8.153900,20.106300,20.106400
12000,2.988500,2.952296,29.579100,8.709300,20.833300,20.837900
14000,2.971700,2.902363,29.953500,9.021500,21.198700,21.215400
16000,2.916100,2.863546,30.362700,9.467300,21.623000,21.628100
18000,2.890200,2.835897,30.744500,9.605100,21.914100,21.921300


Saving model checkpoint to ./checkpoint-10000
/usr/local/lib/python3.7/dist-packages/transformers/configuration_utils.py:337: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  "Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 "
Configuration saved in ./checkpoint-10000/config.json
Model weights saved in ./checkpoint-10000/pytorch_model.bin
tokenizer config file saved in ./checkpoint-10000/tokenizer_config.json
Special tokens file saved in ./checkpoint-10000/special_tokens_map.json
Deleting older checkpoint [checkpoint-4000] due to args.save_total_limit
***** Running Evaluation *****
  Num examples = 3000
  Batch size = 8
Saving model checkpoint to ./checkpoint-12000
/usr/local/lib/python3.7/dist-package

TrainOutput(global_step=19657, training_loss=3.2110016432699706, metrics={'train_runtime': 26310.1329, 'train_samples_per_second': 5.977, 'train_steps_per_second': 0.747, 'total_flos': 1.205836052608512e+17, 'train_loss': 3.2110016432699706, 'epoch': 1.0})

In [20]:
wandb.finish()

eval/loss,█▆▄▃▃▂▂▁▁
eval/rouge1,▁▄▄▆▆▇▇██
eval/rouge2,▁▃▄▅▆▇▇██
eval/rougeL,▁▃▄▆▆▇▇██
eval/rougeLsum,▁▃▄▆▆▇▇██
eval/runtime,█▆▄▁▅▆▅▇▅
eval/samples_per_second,▁▂▄█▄▃▄▂▄
eval/steps_per_second,▁▂▄█▄▃▄▂▄
train/epoch,▁▁▁▂▂▂▂▃▃▃▄▄▄▄▄▅▅▅▅▆▆▆▇▇▇▇▇██
train/global_step,▁▁▁▂▂▂▃▃▃▃▄▄▄▄▄▅▅▅▆▆▆▆▇▇▇▇▇██
train/learning_rate,██▇▇▆▆▆▅▅▄▄▄▃▃▃▂▂▁▁


## Results and Evaluation

In [23]:
!ls -lh

total 2.6G
-rw-r--r-- 1 root root 2.6G Feb 25 01:31 bert2bert-wikihow.zip
drwxr-xr-x 2 root root 4.0K Feb 24 23:19 checkpoint-14000
drwxr-xr-x 2 root root 4.0K Feb 25 00:07 checkpoint-16000
drwxr-xr-x 2 root root 4.0K Feb 25 00:54 checkpoint-18000
drwx------ 5 root root 4.0K Feb 24 17:44 drive
drwxr-xr-x 1 root root 4.0K Feb 18 14:33 sample_data
drwxr-xr-x 3 root root 4.0K Feb 24 17:51 wandb


In [22]:
!zip -r bert2bert-wikihow.zip checkpoint-18000/

  adding: checkpoint-18000/ (stored 0%)
  adding: checkpoint-18000/pytorch_model.bin (deflated 7%)
  adding: checkpoint-18000/rng_state.pth (deflated 27%)
  adding: checkpoint-18000/scheduler.pt (deflated 49%)
  adding: checkpoint-18000/training_args.bin (deflated 48%)
  adding: checkpoint-18000/vocab.txt (deflated 53%)
  adding: checkpoint-18000/trainer_state.json (deflated 78%)
  adding: checkpoint-18000/scaler.pt (deflated 55%)
  adding: checkpoint-18000/tokenizer_config.json (deflated 39%)
  adding: checkpoint-18000/tokenizer.json (deflated 59%)
  adding: checkpoint-18000/optimizer.pt (deflated 10%)
  adding: checkpoint-18000/special_tokens_map.json (deflated 53%)
  adding: checkpoint-18000/config.json (deflated 80%)


In [24]:
!cp bert2bert-wikihow.zip '/content/drive/My Drive/weights/'